In [1]:
import os
import sys
import time
import glob
import numpy as np
import pandas as pd
import cv2

# import warnings
# warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
%pylab inline

sys.path.append("/home/mr/Documents/cell_detection/src")

from utils import label_map_util
from utils import visualization_utils_latest as vis_util

# from my_tracker import centroid_multi_tracker
from my_tracker_with_masks import centroid_multi_tracker

Populating the interactive namespace from numpy and matplotlib
You use ultra mega centroid multitracker


/home/mr/Documents/cell_detection/src/utils/visualization_utils_latest.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-packages

In [2]:
class Video_from_ims:
    def __init__(self, ims_files_list):
        self.files = sorted(ims_files_list)
        f = cv2.imread(ims_files_list[0])
        self.size = (f.shape[1], f.shape[0])
        self.next_id = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.next_id < len(self.files):
            x = self.files[self.next_id]
            self.next_id += 1
            return True, cv2.imread(x)
        return False, None

In [3]:
# dirs_with_ims = glob.glob('../data/interim/for_APB2020/*')

In [4]:
summer_set_dir = '../../Documents/cell_detection/data/interim/jpeg_video/'

In [5]:
labels = np.load('../data/A2_25_full_labels.npy', allow_pickle=True)
labeled_videos = [
 'Well A3_09.avi',
 'Well B3_09.avi',
 'Well A1_09.avi',
 'Well A3_25.avi',
 'Well B1_01.avi',
 'Well B1_17.avi',
 'Well B3_17.avi',
 'Well B1_25.avi',
 'Well B1_09.avi',
 'Well A2_25.avi',
 'Well A2_09.avi',
 'Well B2_01.avi',
 'Well B2_25.avi',
 'Well B3_29.avi',
 'Well B1_05.avi',
 'Well A2_01.avi',
 'Well A1_17.avi',
 'Well A1_25.avi',
 'Well A1_01.avi',
 'Well B2_09.avi',
 'Well A1_29.avi',
 'Well A3_17.avi',
 'Well B3_25.avi',
 'Well A1_21.avi']

In [6]:
%%time
category_index = {'name': 'cell', 'id': 0}
kill_process = False

# for well_id, (pre_vid_name,well_labels) in enumerate(list(zip(labeled_videos,labels))[8:10]):
well_id, pre_vid_name, well_labels = 0, 'Well A2_25.avi', labels[0]
for _ in range(1):
    if kill_process:
        break
    vid_name = summer_set_dir + pre_vid_name
#     well_labels = labels[well_id]
    N_cells = 0
    NNAME = vid_name.split('/')[-1]
    print(NNAME,end=' ')
    video = cv2.VideoCapture(vid_name)
    size = (int(video.get(3)), int(video.get(4)))
    mot_tracker = centroid_multi_tracker(maxLost = 4, max_jump=150, size=size)
#     mot_tracker = OriginalTracker(2)

    # fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#     out_name = vid_name + '.avi'
#     out0 = cv2.VideoWriter('../data/interim/for_APB2020/Well A2_25.avi', fourcc, 2, size)
    out1 = cv2.VideoWriter('../data/interim/for_APB2020/Well A2_25_labeled_maxLost4.avi', fourcc, 3, size)

    ok = True
    j=0
#     try:
    while ok:
        ok, im = video.read()
        if j==58:
            break
        if ok:            
            boxes = well_labels[0][j]
            if len(boxes): # normalisation, cause tracker works with such data
                boxes = boxes / [size[1],size[0],size[1],size[0]]
            scores = well_labels[1][j]
            masks = well_labels[2][j]
#             print(j)
            # filtration that exist in visualisator below
#             boxes = boxes[scores > .7]
#             scores = scores[scores > .7]
#             masks = masks[scores > .7]

#             print(len(boxes),len(scores),len(masks))
            j += 1
            tracked_objects = mot_tracker.update(boxes, masks, scores)
#             print(0.1)
#             (objectID, centroid) in objects.items()
            track_ids = np.array(list(tracked_objects.keys()))
            boxes = np.array([obj['box'] for obj in tracked_objects.values()])
            scores = np.array([obj['score'] for obj in tracked_objects.values()])
            masks = np.array([obj['mask'] for obj in tracked_objects.values()])
            classes = [0]*len(boxes)

#             out0.write(im)
            vis_util.visualize_boxes_and_labels_on_image_array(
                im,
                boxes,
                classes,
                scores,
                category_index, 
                track_ids=track_ids,
                instance_masks=masks,
                use_normalized_coordinates=True,
                line_thickness=1,
                min_score_thresh=.5,
                max_boxes_to_draw=100, 
                skip_labels=True, 
#                     skip_scores=True,
            )
            cv2.putText(im, '{}, {},{}'.format(NNAME[5:-4],j,len(boxes)), 
                        (0,20),cv2.FONT_HERSHEY_SIMPLEX, 0.65, (50,170,50),2)

            out1.write(im)
#             im = cv2.resize(im,(800,600))
#             cv2.imshow('Object detector', im)

#                 if len(boxes) > 50:
#                     break

            if cv2.waitKey(1) == ord('q'):
                kill_process=True
                break
#             time.sleep(.2)
    print('done')
#     except Exception as e:
#         print('fucked | ', e)

#     out0.release()
    out1.release()
#     break
cv2.destroyAllWindows()

Well A2_25.avi done
CPU times: user 20.7 s, sys: 1.12 s, total: 21.8 s
Wall time: 21.7 s


In [7]:
j

58

In [8]:
cv2.destroyAllWindows()

In [9]:
scores

array([0.99768806, 0.99974042, 0.90263331, 0.86509907, 0.98671341,
       0.99633384, 0.99925429, 0.8101697 , 0.98155737, 0.953183  ,
       0.9887861 , 0.98492116, 0.99953389, 0.96661711, 0.86040199,
       0.78902739, 0.64668053, 0.99104261, 0.98104781, 0.80810404,
       0.98404837, 0.96023178, 0.88957918, 0.93743712, 0.99816293,
       0.98126745, 0.71      , 0.78823137, 0.75393242, 0.74522209,
       0.68869728, 0.99664932, 0.62520176, 0.99113911, 0.52282733,
       0.98978245, 0.5145334 , 0.96436644, 0.96163362, 0.95835567,
       0.88517737, 0.8268258 ])

In [10]:
masks.shape

(42, 832, 1114)

In [90]:
for i,x in enumerate(labeled_videos):
    print(i,x)

0 Well A3_09.avi
1 Well B3_09.avi
2 Well A1_09.avi
3 Well A3_25.avi
4 Well B1_01.avi
5 Well B1_17.avi
6 Well B3_17.avi
7 Well B1_25.avi
8 Well B1_09.avi
9 Well A2_25.avi
10 Well A2_09.avi
11 Well B2_01.avi
12 Well B2_25.avi
13 Well B3_29.avi
14 Well B1_05.avi
15 Well A2_01.avi
16 Well A1_17.avi
17 Well A1_25.avi
18 Well A1_01.avi
19 Well B2_09.avi
20 Well A1_29.avi
21 Well A3_17.avi
22 Well B3_25.avi
23 Well A1_21.avi


In [101]:
result = pd.DataFrame(OrderedDict([('bbox',
              {'AP': 26.45479147924208,
               'AP50': 56.36736078338574,
               'AP75': 21.782695678190407,
               'APl': 17.932106190913597,
               'APm': 30.493152743642316,
               'APs': 19.459874356509037}),
             ('segm',
              {'AP': 22.42992617919092,
               'AP50': 57.60324634347454,
               'AP75': 11.82185994451974,
               'APl': 31.025839911941784,
               'APm': 23.405030290852913,
               'APs': 9.609652512672564})])).T

In [102]:
result

,AP,AP50,AP75,APl,APm,APs
bbox,26.454791,56.367361,21.782696,17.932106,30.493153,19.459874
segm,22.429926,57.603246,11.821860,31.025840,23.405030,9.609653


In [ ]:
import cv2 as cv
from scipy.spatial import distance
import numpy as np
from collections import OrderedDict

class OriginalTracker:
    def __init__(self, maxLost = 30):           # maxLost: maximum object lost counted when the object is being tracked
        self.nextObjectID = 0                   # ID of next object
        self.objects = OrderedDict()            # stores ID:Locations
        self.lost = OrderedDict()               # stores ID:Lost_count
        
        self.maxLost = maxLost                  # maximum number of frames object was not detected.
        
    def addObject(self, new_object_location):
        self.objects[self.nextObjectID] = new_object_location    # store new object location
        self.lost[self.nextObjectID] = 0                         # initialize frame_counts for when new object is undetected
        
        self.nextObjectID += 1
    
    def removeObject(self, objectID):                          # remove tracker data after object is lost
        del self.objects[objectID]
        del self.lost[objectID]
    
    @staticmethod
    def getLocation(bounding_box):
        xlt, ylt, xrb, yrb = bounding_box
        return (int((xlt + xrb) / 2.0), int((ylt + yrb) / 2.0))
    
    def update(self,  detections):
        
        if len(detections) == 0:   # if no object detected in the frame
            for objectID in self.lost.keys():
                self.lost[objectID] +=1
                if self.lost[objectID] > self.maxLost: 
                    self.removeObject(objectID)
            
            return self.objects
        
        new_object_locations = np.zeros((len(detections), 2), dtype="int")     # current object locations
        
        for (i, detection) in enumerate(detections): 
            new_object_locations[i] = self.getLocation(detection)
            
        if len(self.objects)==0:
            for i in range(0, len(detections)): 
                self.addObject(new_object_locations[i])
        else:
            objectIDs = list(self.objects.keys())
            previous_object_locations = np.array(list(self.objects.values()))
            
            D = distance.cdist(previous_object_locations, new_object_locations) # pairwise distance between previous and current
            
            row_idx = D.min(axis=1).argsort()   # (minimum distance of previous from current).sort_as_per_index
            
            cols_idx = D.argmin(axis=1)[row_idx]   # index of minimum distance of previous from current
            
            assignedRows, assignedCols = set(), set()
            
            for (row, col) in zip(row_idx, cols_idx):
                
                if row in assignedRows or col in assignedCols:
                    continue
                
                objectID = objectIDs[row]
                self.objects[objectID] = new_object_locations[col]
                self.lost[objectID] = 0
                
                assignedRows.add(row)
                assignedCols.add(col)
                
            unassignedRows = set(range(0, D.shape[0])).difference(assignedRows)
            unassignedCols = set(range(0, D.shape[1])).difference(assignedCols)
            
            
            if D.shape[0]>=D.shape[1]:
                for row in unassignedRows:
                    objectID = objectIDs[row]
                    self.lost[objectID] += 1
                    
                    if self.lost[objectID] > self.maxLost:
                        self.removeObject(objectID)
                        
            else:
                for col in unassignedCols:
                    self.addObject(new_object_locations[col])
            
        return self.objects


## Well A1_08 ___ Well A1_05
- boxes
- scores
- masks

In [10]:
# %%time
# category_index = {'name': 'cell', 'id': 0}

# for well_id,vid_name in enumerate(dirs_with_ims):
#     well_labels = labels[well_id]
#     N_cells = 0
#     NNAME = vid_name.split('/')[-1]
#     print(NNAME + ' ', end='')
#     ims = glob.glob(vid_name + '/*')
#     video = Video_from_ims(ims)
# #     mot_tracker = centroid_multi_tracker(maxLost = 1, max_jump=40, size=size)
#     mot_tracker = OriginalTracker(2)
#     size = video.size
#     # fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
#     fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# #     out_name = vid_name + '.avi'
# #     out = cv2.VideoWriter(out_name, fourcc, 2, size)

#     ok = True
#     j=0
#     while ok:
#         ok, im = next(video)
#         if ok:            
#             boxes = well_labels[0][j]
#             scores = well_labels[1][j]
# #             masks = well_labels[2][j]
#             j += 1
#             print(1)
#             tracked_objects = mot_tracker.update(boxes)
#             print(2)
# #             (objectID, centroid) in objects.items()
#             track_ids = np.array(list(tracked_objects.keys()))
#             boxes = [(objectID, centroid) in objects.items()]
# #             boxes = np.array([obj['box'] for obj in tracked_objects.values()])
# #             scores = np.array([obj['score'] for obj in tracked_objects.values()])
#             classes = [0]*len(boxes)
#             print(3)
#             vis_util.visualize_boxes_and_labels_on_image_array(
#               im,
#               boxes,
#               classes,
#               scores,
#               category_index, 
#               track_ids=track_ids,
# #               instance_masks=masks,
#               use_normalized_coordinates=False,
#               line_thickness=1,
#               min_score_thresh=.7,
#               max_boxes_to_draw=100)

# #             out.write(im)
#             im = cv2.resize(im,(696,520))
            
#             cv2.imshow('Object detector', im)
#             if cv2.waitKey(1) == ord('q'):
#                 kill_process=True
#                 break

# #     out.release()
# #     break
# cv2.destroyAllWindows()

In [ ]:
## Well A1_08 ___ Well A1_05
- boxes
- scores
- masks

# %%time
# category_index = {'name': 'cell', 'id': 0}

# for well_id,vid_name in enumerate(dirs_with_ims):
#     well_labels = labels[well_id]
#     N_cells = 0
#     NNAME = vid_name.split('/')[-1]
#     print(NNAME + ' ', end='')
#     ims = glob.glob(vid_name + '/*')
#     video = Video_from_ims(ims)
# #     mot_tracker = centroid_multi_tracker(maxLost = 1, max_jump=40, size=size)
#     mot_tracker = OriginalTracker(2)
#     size = video.size
#     # fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
#     fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# #     out_name = vid_name + '.avi'
# #     out = cv2.VideoWriter(out_name, fourcc, 2, size)

#     ok = True
#     j=0
#     while ok:
#         ok, im = next(video)
#         if ok:            
#             boxes = well_labels[0][j]
#             scores = well_labels[1][j]
# #             masks = well_labels[2][j]
#             j += 1
#             print(1)
#             tracked_objects = mot_tracker.update(boxes)
#             print(2)
# #             (objectID, centroid) in objects.items()
#             track_ids = np.array(list(tracked_objects.keys()))
#             boxes = [(objectID, centroid) in objects.items()]
# #             boxes = np.array([obj['box'] for obj in tracked_objects.values()])
# #             scores = np.array([obj['score'] for obj in tracked_objects.values()])
#             classes = [0]*len(boxes)
#             print(3)
#             vis_util.visualize_boxes_and_labels_on_image_array(
#               im,
#               boxes,
#               classes,
#               scores,
#               category_index, 
#               track_ids=track_ids,
# #               instance_masks=masks,
#               use_normalized_coordinates=False,
#               line_thickness=1,
#               min_score_thresh=.7,
#               max_boxes_to_draw=100)

# #             out.write(im)
#             im = cv2.resize(im,(696,520))
            
#             cv2.imshow('Object detector', im)
#             if cv2.waitKey(1) == ord('q'):
#                 kill_process=True
#                 break

# #     out.release()
# #     break
# cv2.destroyAllWindows()